In [1]:
import tensorflow as tf

2022-02-22 15:38:58.464222: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

2022-02-22 15:39:04.320082: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-22 15:39:04.323021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-22 15:39:04.350677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-22 15:39:04.351263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-02-22 15:39:04.351287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-02-22 15:39:04.354236: I tensorflow/stream_executor/platform/default/dso_loade

In [9]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os, shutil

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets

In [10]:
# konlpy 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 106.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.0/449.0 KB 60.4 MB/s eta 0:00:00


In [65]:
%%shell
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

UsageError: Cell magic `%%shell` not found.


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [48]:
train=pd.read_csv('/home/lab12/data/train_data.csv')
train

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


In [55]:
test=pd.read_csv('/home/lab12/data/test_data.csv')
test

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


In [56]:
del test['index']

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45654 entries, 0 to 45653
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      45654 non-null  object
 1   topic_idx  45654 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 713.5+ KB


In [57]:
test1=test.values.tolist()
test1[:2]

[['유튜브 내달 2일까지 크리에이터 지원 공간 운영'], ['어버이날 맑다가 흐려져…남부지방 옅은 황사']]

In [27]:
from tensorflow.keras.datasets import imdb

In [50]:
train1=train.values.tolist()
train1[:2]

[[0, '인천→핀란드 항공기 결항…휴가철 여행객 분통', 4], [1, '실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화', 4]]

In [49]:
len(train1)

45654

In [51]:
import re

In [52]:
def cleanText( text ):
    ret = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", '', text)
    return ret

In [53]:
print( train1[11] )
cleanText(train1[11][1])

[11, '이란서 최고지도자 모욕 혐의 미국인에 징역 10년 선고', 4]


'이란서 최고지도자 모욕 혐의 미국인에 징역 년 선고'

In [54]:
train_clean = []
for line in train1:
    train_clean.append( [ cleanText(line[1]), int(line[2])] )
train_clean[:2]

[['인천핀란드 항공기 결항휴가철 여행객 분통', 4], ['실리콘밸리 넘어서겠다구글 조원 들여 전역 거점화', 4]]

In [59]:
%cd '/home/lab12/data'
!wget https://bab2min.tistory.com/attachment/cfile2.uf@241D6F475873C2B1010DEA.txt
!mv cfile2.uf@241D6F475873C2B1010DEA.txt stopwords.txt

/home/lab12/data
--2022-02-22 17:15:04--  https://bab2min.tistory.com/attachment/cfile2.uf@241D6F475873C2B1010DEA.txt
Resolving bab2min.tistory.com (bab2min.tistory.com)... 211.249.222.33
Connecting to bab2min.tistory.com (bab2min.tistory.com)|211.249.222.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://t1.daumcdn.net/cfile/tistory/241D6F475873C2B101?download [following]
--2022-02-22 17:15:04--  https://t1.daumcdn.net/cfile/tistory/241D6F475873C2B101?download
Resolving t1.daumcdn.net (t1.daumcdn.net)... 23.33.33.201, 23.33.33.177, 2600:140b:400::1721:208b, ...
Connecting to t1.daumcdn.net (t1.daumcdn.net)|23.33.33.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2171 (2.1K) [text/plain]
Saving to: ‘cfile2.uf@241D6F475873C2B1010DEA.txt’

cfile2.uf@241D6F475 100%[===================>]   2.12K  --.-KB/s    in 0s      

2022-02-22 17:15:04 (406 MB/s) - ‘cfile2.uf@241D6F475873C2B1010DEA.txt’ saved [2171/2171]



In [60]:
stopwords = []
with open('./stopwords.txt') as f:
    stopwords = [ line.split('\t')[0] for line in f.readlines() ]
stopwords[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [62]:
# 한글에서 사용할 수 있는 형태소 분석기
from konlpy.tag import Kkma, Komoran, Okt, Hannanum, Mecab

In [66]:
kkma = Kkma()
komoran = Komoran()
han = Hannanum()

okt = Okt()

In [67]:
print( okt.morphs( train_clean[0][0] ) )
print( okt.morphs( train_clean[0][0], stem=True ) )
print( okt.morphs( train_clean[0][0], stem=True, norm=True ) )

['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통']
['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통']
['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통']


In [68]:
train_token = []
for line in train_clean:
    pre = [word for word in okt.morphs( line[0], stem=True, norm=True ) if word not in stopwords and len(word) > 1]
    train_token.append( [pre, line[1]] )

In [70]:
train_token[:2]

[[['인천', '핀란드', '항공기', '결항', '휴가', '여행객', '분통'], 4],
 [['실리콘밸리', '넘어서다', '구글', '조원', '들이다', '전역', '거점'], 4]]

In [71]:
import pickle

In [72]:
!pwd

/home/lab12/data


In [73]:
with open('./nsmc_train_preprocess.pk', 'wb') as f:
    pickle.dump(train_token, f)